# 🧠 om-memory vs Traditional RAG — Live Benchmark

**om-memory** is a Python library that gives AI agents human-like long-term memory.
Instead of stuffing the entire conversation history into every API call (like RAG),
it compresses old messages into dense observations — keeping context small and costs low.

In this notebook, we'll:
1. **Build a Traditional RAG chatbot** using ChromaDB + OpenAI
2. **Build the same chatbot with om-memory**
3. **Run them side-by-side** for 20 turns and compare token usage
4. **Test memory accuracy** — can om-memory recall facts after compression?

> 💡 You only need an **OpenAI API key** to run this notebook.

## Step 1: Install Dependencies

In [ ]:
!pip install -q om-memory openai chromadb matplotlib numpy

## Step 2: Enter Your API Key

In [ ]:
import os
from getpass import getpass

# Enter your OpenAI API key (it won't be displayed)
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
print("✅ API key set!")

## Step 3: Set Up the Knowledge Base with ChromaDB

We'll create a small company handbook as our knowledge base and index it in ChromaDB.
This simulates a real RAG setup.

In [ ]:
import chromadb

# Company handbook chunks
KB_CHUNKS = [
    "PTO POLICY: Full-time employees get 20 PTO days per year, accruing at 1.67 days per month from day one. Unused PTO carries over up to 5 days. Requests need 2 weeks advance notice for trips over 3 days. Q4 peak season: max 3 consecutive days unless VP-approved. Employees with 5+ years tenure get 25 days total.",
    "REMOTE WORK: Hybrid model — engineers remote on Tuesday and Thursday. Senior staff (L5+) can work up to 3 days per week remotely with manager approval. Core hours are 10AM-4PM EST. VPN is required for all remote work. Home office stipend is $1,500 per year (no rollover). Daily standup at 10:15AM EST is mandatory.",
    "TRAVEL EXPENSES: Meals $50/day domestic, $75/day international. Hotels $200/night domestic, $300/night international. Submit expense reports within 30 days via Expensify. Receipts required for expenses over $25. Economy class for domestic flights; business class for international flights over 6 hours. Mileage reimbursement at $0.67/mile.",
    "BENEFITS: 401k match up to 6% of salary. Health insurance options: Aetna PPO or Kaiser HMO. Dental and vision included. Life insurance at 2x annual salary. Employee Assistance Program (EAP) available 24/7. Gym membership reimbursement up to $75/month.",
    "PROFESSIONAL DEVELOPMENT: $3,000/year learning budget for courses, books, and subscriptions. Conference attendance requires manager approval 30 days in advance. Professional certifications are reimbursed 100% if passed on first attempt."
]

# Create ChromaDB collection
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="handbook")
collection.add(
    documents=KB_CHUNKS,
    ids=[f"chunk_{i}" for i in range(len(KB_CHUNKS))]
)
print(f"✅ Indexed {len(KB_CHUNKS)} knowledge base chunks in ChromaDB")

## Step 4: Define the Chat Functions

We'll create two chat functions:
- **`chat_rag()`** — Traditional RAG: retrieves relevant chunks + sends full conversation history
- **`chat_om()`** — om-memory: retrieves chunks + sends compressed observations + recent messages

In [ ]:
from openai import AsyncOpenAI
from om_memory import ObservationalMemory, OMConfig

oai = AsyncOpenAI()

async def chat_rag(history, query):
    """Traditional RAG: ChromaDB retrieval + full conversation history."""
    # Retrieve relevant KB chunks
    results = collection.query(query_texts=[query], n_results=2)
    kb_context = "\n".join(results['documents'][0])
    
    # Build prompt with FULL history (this grows linearly!)
    history_text = "\n".join([f"{m['role']}: {m['content']}" for m in history])
    system_prompt = (
        f"You are an HR assistant. Answer concisely from this knowledge base:\n"
        f"{kb_context}\n\nConversation history:\n{history_text}"
    )
    
    resp = await oai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": query}],
        max_tokens=100,
    )
    answer = resp.choices[0].message.content
    history.extend([{"role": "user", "content": query}, {"role": "assistant", "content": answer}])
    return answer, resp.usage.prompt_tokens, resp.usage.completion_tokens


async def chat_om(om, thread_id, query):
    """om-memory: ChromaDB retrieval + compressed observations + recent messages."""
    # Retrieve relevant KB chunks
    results = collection.query(query_texts=[query], n_results=2)
    kb_context = "\n".join(results['documents'][0])
    
    # Get compressed context from om-memory (observations + last few messages)
    memory_ctx = await om.aget_context(thread_id)
    
    system_prompt = (
        f"You are an HR assistant. Answer concisely from this knowledge base:\n"
        f"{kb_context}\n\n{memory_ctx}"
    )
    
    resp = await oai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": query}],
        max_tokens=100,
    )
    answer = resp.choices[0].message.content
    await om.aadd_message(thread_id, "user", query)
    await om.aadd_message(thread_id, "assistant", answer)
    return answer, resp.usage.prompt_tokens, resp.usage.completion_tokens

print("✅ Chat functions defined")

## Step 5: Run the 20-Turn Comparison

Now we run both approaches through the same 20 questions and track token usage.

In [ ]:
import time

QUERIES = [
    "Hi, I'm Alex Chen. I'm an L5 senior engineer, been here 6 years.",
    "How many PTO days do I get with my tenure?",
    "Can I work remotely 3 days a week?",
    "What's my home office stipend?",
    "I want to attend KubeCon in Paris. What's the process?",
    "The flight to Paris is 8 hours. Can I fly business class?",
    "What's the international hotel allowance?",
    "What about meal expenses in Paris?",
    "Do I need receipts for a 20 euro coffee?",
    "When do I submit expense reports?",
    "Can I use my personal credit card?",
    "I want a week of vacation after KubeCon. How do I request it?",
    "How many carry-over days from last year?",
    "What's the mileage rate to the airport?",
    "Is VPN required from the Paris hotel?",
    "What are the core remote work hours?",
    "My manager wants to know Q4 travel restrictions.",
    "What certifications does the company reimburse?",
    "What's my gym reimbursement amount?",
    "Give me a summary of my complete travel plan and benefits.",
]

# --- Run Traditional RAG ---
print("🔴 Running Traditional RAG...")
rag_history = []
rag_tokens = []
rag_total = 0

for i, q in enumerate(QUERIES):
    answer, pt, ct = await chat_rag(rag_history, q)
    rag_total += pt + ct
    rag_tokens.append(pt)
    print(f"  Turn {i+1:2d}: prompt={pt:5d} tokens")

# --- Run om-memory ---
print("\n🟢 Running om-memory...")
config = OMConfig(
    observer_token_threshold=300,
    reflector_token_threshold=1500,
    message_retention_count=2,
    message_token_budget=200,
    auto_observe=True,
    auto_reflect=True,
    blocking_mode=True,
)
om = ObservationalMemory(api_key=os.environ["OPENAI_API_KEY"], config=config)
await om.ainitialize()
thread_id = f"colab_{int(time.time())}"

om_tokens = []
om_total = 0

for i, q in enumerate(QUERIES):
    answer, pt, ct = await chat_om(om, thread_id, q)
    om_total += pt + ct
    om_tokens.append(pt)
    obs_count = len(await om.aget_observations(thread_id))
    print(f"  Turn {i+1:2d}: prompt={pt:5d} tokens  |  observations={obs_count}")

stats = await om.aget_stats(thread_id)
bg_tokens = stats.total_input_tokens + stats.total_output_tokens
om_total_with_bg = om_total + bg_tokens

print(f"\n📊 Results:")
print(f"  RAG total:    {rag_total:,} tokens")
print(f"  OM total:     {om_total_with_bg:,} tokens (incl. {bg_tokens:,} background)")
savings = ((rag_total - om_total_with_bg) / rag_total) * 100
print(f"  Savings:      {savings:.1f}%")

## Step 6: Visualize the Results

Let's plot the token usage per turn — you'll see RAG growing linearly while om-memory stays flat.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Dark theme
plt.rcParams.update({
    'figure.facecolor': '#0d1117', 'axes.facecolor': '#161b22',
    'axes.edgecolor': '#30363d', 'axes.labelcolor': '#e6edf3',
    'text.color': '#e6edf3', 'xtick.color': '#8b949e',
    'ytick.color': '#8b949e', 'grid.color': '#21262d',
})

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
turns = range(1, len(rag_tokens) + 1)

# Chart 1: Per-turn tokens
ax1.plot(turns, rag_tokens, color='#f85149', linewidth=2.5, label='Traditional RAG', marker='o', markersize=4)
ax1.fill_between(turns, rag_tokens, alpha=0.15, color='#f85149')
ax1.plot(turns, om_tokens, color='#3fb950', linewidth=2.5, label='om-memory', marker='s', markersize=4)
ax1.fill_between(turns, om_tokens, alpha=0.15, color='#3fb950')
ax1.set_xlabel('Turn', fontsize=12)
ax1.set_ylabel('Prompt Tokens', fontsize=12)
ax1.set_title('Per-Turn Prompt Tokens', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Chart 2: Cumulative
rag_cumul = np.cumsum(rag_tokens)
om_cumul = np.cumsum(om_tokens)
ax2.plot(turns, rag_cumul, color='#f85149', linewidth=2.5, label='RAG (cumulative)')
ax2.plot(turns, om_cumul, color='#3fb950', linewidth=2.5, label='OM (cumulative)')
ax2.fill_between(turns, om_cumul, rag_cumul, alpha=0.15, color='#3fb950', label='Savings')
ax2.set_xlabel('Turn', fontsize=12)
ax2.set_ylabel('Cumulative Prompt Tokens', fontsize=12)
ax2.set_title('Cumulative Token Usage', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 By turn 20: RAG uses {rag_tokens[-1]:,} prompt tokens vs OM uses {om_tokens[-1]:,}")
print(f"   That's a {((rag_tokens[-1] - om_tokens[-1]) / rag_tokens[-1] * 100):.0f}% reduction per turn!")

## Step 7: Test Memory Accuracy

The most critical question: **Does om-memory actually remember things?**

After compression, we'll ask the agent to recall facts from early in the conversation.

In [ ]:
recall_tests = [
    ("What is my name?", ["Alex", "Chen"]),
    ("What level am I?", ["L5", "senior"]),
    ("How long have I been at the company?", ["6"]),
    ("What conference am I planning to attend?", ["KubeCon"]),
    ("What's my gym reimbursement?", ["75"]),
]

print("🧪 Memory Accuracy Test")
print("(Testing if om-memory recalls facts from early turns after compression)\n")

passed = 0
for question, keywords in recall_tests:
    memory_ctx = await om.aget_context(thread_id)
    results = collection.query(query_texts=[question], n_results=2)
    kb_context = "\n".join(results['documents'][0])
    
    resp = await oai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": f"You are an HR assistant.\nKB: {kb_context}\n{memory_ctx}"},
            {"role": "user", "content": question}
        ],
        max_tokens=80,
    )
    answer = resp.choices[0].message.content
    found = any(kw.lower() in answer.lower() for kw in keywords)
    if found: passed += 1
    print(f"  {'✅' if found else '❌'} {question}")
    print(f"     → {answer[:80]}")

accuracy = (passed / len(recall_tests)) * 100
print(f"\n📊 Memory Accuracy: {passed}/{len(recall_tests)} ({accuracy:.0f}%)")

## Step 8: See What om-memory Stored

Let's peek inside om-memory to see the compressed observations it created.

In [ ]:
observations = await om.aget_observations(thread_id)
messages = await om.storage.aget_messages(thread_id)

print(f"📦 om-memory state:")
print(f"  Observations: {len(observations)} (compressed from 20 turns)")
print(f"  Messages retained: {len(messages)} (rolling window)")
print(f"  Background tokens used: {bg_tokens:,}\n")

print("🔍 Stored Observations:")
for obs in observations:
    print(f"  {obs.priority.value} {obs.content}")
    
await om.aclose()

## How It Works: Traditional RAG vs om-memory

| | Traditional RAG | om-memory |
|---|---|---|
| **Context growth** | Linear — O(n) per turn | Flat — O(1) per turn |
| **Memory mechanism** | Full history in every call | Compressed observations |
| **Token cost** | Grows with conversation | Stays stable |
| **Accuracy** | 100% (has everything) | High (95%+ in testing) |
| **Long conversations** | Hits context limit | Unlimited |

### Architecture
```
Traditional RAG:    [System Prompt] + [KB Chunks] + [ALL Messages]
                                                    ^^^^^^^^^^^^^^^^
                                                    grows every turn!

om-memory:          [System Prompt] + [KB Chunks] + [Observations] + [Last 2 Messages]
                                                    ^^^^^^^^^^^^^^   ^^^^^^^^^^^^^^^^^
                                                    stable/cached    tiny rolling window
```

### Install om-memory
```bash
pip install om-memory
```

📦 [PyPI](https://pypi.org/project/om-memory/) · 🐙 [GitHub](https://github.com/om-memory/om-memory)